In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

region = boto3.Session().region_name

s3_client = boto3.client("s3")

sess = sagemaker.Session()

output_bucket = sess.default_bucket()
output_prefix = "ic-transfer-learning"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
arn:aws:iam::324183265896:role/service-role/AmazonSageMaker-ExecutionRole-20250604T045982


In [ ]:
# Generate .lst file from JSON

import json
import os

# Load JSON data
with open("grp4-metadata.json", "r") as f:
    data = json.load(f)

# Create label-to-index mapping
# categories = list(set([item["category_id"] for item in data]))
# label_map = {category: idx for idx, category in enumerate(sorted(categories))}

# Split into train/val and write .lst files
for split_type in ["train", "val"]:
    split_data = [item for item in data if item["split_type"] == split_type]
    
    with open(f"{split_type}.lst", "w") as f:
        for idx, item in enumerate(split_data):
            relative_path = item["filename"]
            category_id = item["category_id"]
            f.write(f"{idx}\t{category_id}\t{relative_path}\n")

In [2]:
import boto3

src_bucket = "aai-540-data"
src_prefix = "cct_resized"

s3_client.upload_file(
    "train.lst", src_bucket, src_prefix + "/train_lst/train.lst"
)

In [3]:
s3_client.upload_file(
    "val.lst", src_bucket, src_prefix + "/val_lst/train.lst"
)

In [4]:
# retrieve base SageMakers image-classification model 
from sagemaker import image_uris

training_image = image_uris.retrieve(
    "image-classification", sess.boto_region_name, version="latest"
)

print(training_image)

811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:1


In [5]:
# Configure input channels
input_data = {
    "train": sagemaker.inputs.TrainingInput(
        s3_data="s3://aai-540-data/cct_resized/",  # All images here
        content_type="application/x-image",
    ),
    "validation": sagemaker.inputs.TrainingInput(
        s3_data="s3://aai-540-data/cct_resized/",  # Same directory
        content_type="application/x-image",
    ),
    "train_lst": sagemaker.inputs.TrainingInput(
        s3_data="s3://aai-540-data/cct_resized/train_lst/",
        content_type="application/x-image",
    ),
    "validation_lst": sagemaker.inputs.TrainingInput(
        s3_data="s3://aai-540-data/cct_resized/val_lst/",
        content_type="application/x-image",
    ),
}

In [6]:
# Configure base image classifier
s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"
ic_estimator = sagemaker.estimator.Estimator(
    training_image,
    role,
    instance_count=1,
    instance_type="ml.g4dn.xlarge",
    volume_size=50,
    max_run=360000,
    input_mode="File",
    output_path=s3_output_location,
    sagemaker_session=sess,
)

In [7]:
# Configure hyper parameters
ic_estimator.set_hyperparameters(
    num_layers=18,
    use_pretrained_model=1,
    image_shape="3,224,224",
    num_classes=20,
    num_training_samples=39360,
    mini_batch_size=128,
    epochs=2,
    learning_rate=0.01,
    precision_dtype="float32",
)

In [8]:
ic_estimator.fit(inputs=input_data, logs=True)

INFO:sagemaker:Creating training-job with name: image-classification-2025-06-10-12-26-18-032


2025-06-10 12:26:22 Starting - Starting the training job...
2025-06-10 12:26:37 Starting - Preparing the instances for training...
2025-06-10 12:27:08 Downloading - Downloading input data.........
2025-06-10 12:28:54 Downloading - Downloading the training image.........
2025-06-10 12:30:00 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
Nvidia gpu devices, drivers and cuda toolkit versions (only available on hosts with GPU):
Tue Jun 10 12:30:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           